In [ ]:
# 필요한 라이브러리 설치
!pip install openai faiss-cpu numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.8 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import faiss
from openai import OpenAI
import pandas as pd
from collections import defaultdict

# OpenAI API 키 설정
client = OpenAI(api_key=)

In [ ]:
# ===========================
# 유틸리티 함수
# ===========================

def load_jsonl(file_path):
    """JSONL 파일을 로드하여 리스트로 반환"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def get_embedding(text, model="text-embedding-3-small"):
    """OpenAI API를 사용하여 텍스트 임베딩"""
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

def normalize_embedding(embedding):
    """L2 정규화"""
    embedding = np.array(embedding).astype('float32')
    norm = np.linalg.norm(embedding)
    if norm > 0:
        embedding = embedding / norm
    return embedding.reshape(1, -1)

def search_similar_projects(query_embedding, index, k=20):
    """FAISS 인덱스에서 유사한 프로젝트 검색"""
    query_embedding = normalize_embedding(query_embedding)
    distances, indices = index.search(query_embedding, k)
    return indices[0], distances[0]

# ===========================
# 3단계: 추천 시스템 구현
# ===========================

def recommend_projects(user_id, top_k=5):
    """
    user_id를 입력받아 콘텐츠 기반 필터링으로 프로젝트 추천

    Args:
        user_id: 추천받을 사용자 ID
        top_k: 최종 추천할 프로젝트 개수 (기본값: 5)

    Returns:
        추천된 프로젝트 ID 리스트와 점수
    """

    # 1. 데이터 로드
    print("데이터 로딩 중...")
    users = load_jsonl('user_textified.jsonl')
    projects = load_jsonl('project_textified.jsonl')

    # 2. FAISS 인덱스 로드
    print("FAISS 인덱스 로딩 중...")
    index_text = faiss.read_index('project_text.index')
    index_skill = faiss.read_index('project_skill.index')
    index_role = faiss.read_index('project_role.index')
    index_field = faiss.read_index('project_field.index')

    # 3. 해당 유저 정보 추출
    user = None
    for u in users:
        if u['user_id'] == user_id:
            user = u
            break

    if user is None:
        print(f"User {user_id}를 찾을 수 없습니다.")
        return None

    print(f"\n사용자 정보:")
    print(f"- ID: {user['user_id']}")
    print(f"- 이름: {user['name']}")
    print(f"- Text: {user['u_text'][:100]}...")
    print(f"- Skill: {user['u_skill']}")
    print(f"- Role: {user['u_role']}")
    print(f"- Interest: {user['u_interest']}")

    # 4. 유저 정보 임베딩
    print("\n유저 정보 임베딩 중...")
    emb_text = get_embedding(user['u_text'])
    emb_skill = get_embedding(user['u_skill'])
    emb_role = get_embedding(user['u_role'])
    emb_interest = get_embedding(user['u_interest'])

    # 5. 각 카테고리별로 유사한 프로젝트 검색 (top-20)
    print("\n유사 프로젝트 검색 중...")
    indices_text, scores_text = search_similar_projects(emb_text, index_text, k=100)
    indices_skill, scores_skill = search_similar_projects(emb_skill, index_skill, k=100)
    indices_role, scores_role = search_similar_projects(emb_role, index_role, k=100)
    indices_field, scores_field = search_similar_projects(emb_interest, index_field, k=100)

    print(f"\n{'='*80}")
    print("카테고리별 검색 결과 (Top-20)")
    print(f"{'='*80}")

    # Text 카테고리 결과
    print(f"\n[TEXT 카테고리]")
    for rank, (idx, score) in enumerate(zip(indices_text, scores_text), 1):
        project = projects[idx]
        print(f"{rank}. {project['project_id']} (유사도: {score:.4f})")
        print(f"   {project['p_text'][:80]}...")
        print()

    # Skill 카테고리 결과
    print(f"\n[SKILL 카테고리]")
    for rank, (idx, score) in enumerate(zip(indices_skill, scores_skill), 1):
        project = projects[idx]
        print(f"{rank}. {project['project_id']} (유사도: {score:.4f})")
        print(f"   {project['p_skill']}")
        print()

    # Role 카테고리 결과
    print(f"\n[ROLE 카테고리]")
    for rank, (idx, score) in enumerate(zip(indices_role, scores_role), 1):
        project = projects[idx]
        print(f"{rank}. {project['project_id']} (유사도: {score:.4f})")
        print(f"   {project['p_role']}")
        print()

    # Field 카테고리 결과
    print(f"\n[FIELD 카테고리]")
    for rank, (idx, score) in enumerate(zip(indices_field, scores_field), 1):
        project = projects[idx]
        print(f"{rank}. {project['project_id']} (유사도: {score:.4f})")
        print(f"   {project['p_field']}")
        print()

    # 6. 최종 점수 계산
    print("\n최종 점수 계산 중...")
    project_scores = defaultdict(lambda: {'text': 0, 'skill': 0, 'role': 0, 'field': 0})

    # text 유사도 점수 저장
    for idx, score in zip(indices_text, scores_text):
        project_scores[idx]['text'] = float(score)

    # skill 유사도 점수 저장
    for idx, score in zip(indices_skill, scores_skill):
        project_scores[idx]['skill'] = float(score)

    # role 유사도 점수 저장
    for idx, score in zip(indices_role, scores_role):
        project_scores[idx]['role'] = float(score)

    # field 유사도 점수 저장
    for idx, score in zip(indices_field, scores_field):
        project_scores[idx]['field'] = float(score)

    # CBF 최종 점수 계산
    final_scores = {}
    for project_idx, scores in project_scores.items():
        cbf_score = (0.40 * scores['text'] +
                     0.30 * scores['skill'] +
                     0.20 * scores['role'] +
                     0.10 * scores['field'])
        final_scores[project_idx] = {
            'total': cbf_score,
            'details': scores
        }

    # 7. 상위 k개 프로젝트 선택
    sorted_projects = sorted(final_scores.items(), key=lambda x: x[1]['total'], reverse=True)
    top_projects = sorted_projects[:top_k]

    # 8. 결과 출력
    print(f"\n{'='*80}")
    print(f"추천 결과 (Top {top_k})")
    print(f"{'='*80}")

    recommended_project_ids = []
    for rank, (project_idx, score_info) in enumerate(top_projects, 1):
        project = projects[project_idx]
        recommended_project_ids.append(project['project_id'])

        print(f"\n[{rank}위] {project['project_id']}")
        print(f"제목/설명: {project['p_text'][:100]}...")
        print(f"필요 스킬: {project['p_skill']}")
        print(f"필요 역할: {project['p_role']}")
        print(f"마감일: {project['deadline']}")
        print(f"점수: {score_info['total']:.4f}")
        print(f"  - text: {score_info['details']['text']:.4f} (가중치 40%)")
        print(f"  - skill: {score_info['details']['skill']:.4f} (가중치 30%)")
        print(f"  - role: {score_info['details']['role']:.4f} (가중치 20%)")
        print(f"  - field: {score_info['details']['field']:.4f} (가중치 10%)")

    return recommended_project_ids, top_projects


In [ ]:
# ===========================
# 4단계: 평가 지표 계산
# ===========================

def build_positive_set(user_id, top_n=30):
    """
    user와 project 간 교집합 기준으로 positive set 구축

    Args:
        user_id: 사용자 ID
        top_n: positive로 간주할 상위 프로젝트 개수

    Returns:
        positive set (프로젝트 ID 리스트)
    """
    print(f"\n{'='*80}")
    print(f"Positive Set 구축")
    print(f"{'='*80}")

    # 데이터 로드
    users = load_jsonl('user_textified.jsonl')
    projects = load_jsonl('project_textified.jsonl')

    # 해당 유저 찾기
    user = None
    for u in users:
        if u['user_id'] == user_id:
            user = u
            break

    if user is None:
        print(f"User {user_id}를 찾을 수 없습니다.")
        return None

    # 유저 정보에서 키워드 추출 (간단하게 공백 기준 분리)
    def extract_keywords(text):
        """텍스트에서 키워드 추출"""
        # "skills: Python, Django" 같은 형식에서 실제 값만 추출
        if ':' in text:
            text = text.split(':', 1)[1]
        keywords = [k.strip().lower() for k in text.replace(',', ' ').split()]
        return set(keywords)

    user_skills = extract_keywords(user['u_skill'])
    user_role = extract_keywords(user['u_role'])
    user_interests = extract_keywords(user['u_interest'])
    user_text_keywords = extract_keywords(user['u_text'])

    print(f"\n사용자 키워드:")
    print(f"- Skills: {user_skills}")
    print(f"- Role: {user_role}")
    print(f"- Interests: {user_interests}")

    # 모든 프로젝트에 대해 overlap 점수 계산
    project_overlaps = []

    for project in projects:
        project_skills = extract_keywords(project['p_skill'])
        project_roles = extract_keywords(project['p_role'])
        project_fields = extract_keywords(project['p_field'])
        project_text_keywords = extract_keywords(project['p_text'])

        # 교집합 개수 계산
        skill_overlap = len(user_skills & project_skills)
        role_overlap = len(user_role & project_roles)
        field_overlap = len(user_interests & project_fields)
        text_overlap = len(user_text_keywords & project_text_keywords)

        # 총 overlap 점수 (단순 합산)
        total_overlap = skill_overlap + role_overlap + field_overlap + text_overlap

        project_overlaps.append({
            'project_id': project['project_id'],
            'project': project,
            'overlap_score': total_overlap,
            'details': {
                'skill': skill_overlap,
                'role': role_overlap,
                'field': field_overlap,
                'text': text_overlap
            }
        })

    # overlap 점수 기준으로 정렬
    project_overlaps.sort(key=lambda x: x['overlap_score'], reverse=True)

    # 상위 top_n개를 positive set으로 선택
    positive_set = [p['project_id'] for p in project_overlaps[:top_n]]

    print(f"\nPositive Set (상위 {top_n}개):")
    for i, p in enumerate(project_overlaps[:top_n], 1):
        print(f"{i}. {p['project_id']} - Overlap: {p['overlap_score']} "
              f"(skill:{p['details']['skill']}, role:{p['details']['role']}, "
              f"field:{p['details']['field']}, text:{p['details']['text']})")

    return positive_set

def evaluate_recommendations(user_id, recommended_ids, positive_set):
    """
    추천 결과를 평가 (Hit Rate 계산)

    Args:
        user_id: 사용자 ID
        recommended_ids: 추천된 프로젝트 ID 리스트
        positive_set: positive로 간주되는 프로젝트 ID 리스트

    Returns:
        hit_rate 값
    """
    print(f"\n{'='*80}")
    print(f"평가 결과")
    print(f"{'='*80}")

    # Hit: 추천된 것 중 positive set에 있는 것
    hits = set(recommended_ids) & set(positive_set)

    # Hit Rate = (추천된 것 중 positive에 포함된 개수) / (추천 개수)
    hit_rate = len(hits) / len(recommended_ids) if len(recommended_ids) > 0 else 0

    print(f"\n추천된 프로젝트: {recommended_ids}")
    print(f"Positive Set 크기: {len(positive_set)}")
    print(f"Hit 프로젝트: {list(hits)}")
    print(f"\n📊 평가 지표:")
    print(f"  - Hit Rate@{len(recommended_ids)}: {hit_rate:.4f} ({len(hits)}/{len(recommended_ids)})")
    print(f"  - 해석: 추천한 {len(recommended_ids)}개 중 {len(hits)}개가 Positive Set에 포함됨")

    if hit_rate == 1.0:
        print(f"  - 🎉 완벽한 추천! 모든 추천이 Positive Set에 포함되었습니다!")
    elif hit_rate >= 0.6:
        print(f"  - ✅ 좋은 추천 성능입니다!")
    elif hit_rate >= 0.4:
        print(f"  - ⚠️ 보통 수준의 추천 성능입니다.")
    else:
        print(f"  - ❌ 추천 성능 개선이 필요합니다.")

    return hit_rate

In [ ]:
if __name__ == "__main__":
    # 평가할 사용자 ID 설정
    target_user_id = "U00001"  # 여기에 실제 user_id를 입력하세요

    print("="*80)
    print("콘텐츠 기반 필터링 추천 시스템")
    print("="*80)

    # 3단계: 프로젝트 추천
    recommended_ids, top_projects = recommend_projects(target_user_id, top_k=5)

    # 4단계: Positive set 구축 및 평가
    positive_set = build_positive_set(target_user_id, top_n=30)

    # 평가 지표 계산
    hit_rate = evaluate_recommendations(target_user_id, recommended_ids, positive_set)

    print(f"\n{'='*80}")
    print("✅ 추천 및 평가 완료!")
    print(f"{'='*80}")

콘텐츠 기반 필터링 추천 시스템
데이터 로딩 중...
FAISS 인덱스 로딩 중...

사용자 정보:
- ID: U00001
- 이름: 이수현
- Text: 데이터 기반의 AI 솔루션 개발자가 되려 합니다. 웹 애플리케이션 개발 백엔드 API 설계 및 구현...
- Skill: skills: Python, Django, React, Docker, AWS
- Role: preferred role: 풀스택
- Interest: major: 컴퓨터공학, 데이터사이언스 interests: 자연어 처리, 클라우드 인프라, API 설계

유저 정보 임베딩 중...

유사 프로젝트 검색 중...

카테고리별 검색 결과 (Top-20)

[TEXT 카테고리]
1. p_1164 (유사도: 0.6040)
   AI 기반 프라이버시 보호 시스템 개발. 사용자의 개인정보를 안전하게 보호하는 AI 시스템을 설계하고 구현합니다....

2. p_1203 (유사도: 0.5941)
   AI 기반 프라이버시 보호 시스템 설계. 사용자의 데이터 프라이버시를 보장하는 AI 시스템을 설계합니다....

3. p_2237 (유사도: 0.5916)
   AI 기반 추천 시스템 개발. 사용자 맞춤형 콘텐츠 추천을 위한 AI 모델 설계 및 구현을 진행합니다....

4. p_0099 (유사도: 0.5904)
   AI 기반 추천시스템 개발. 사용자 개인화 경험을 제공하기 위해 AI 추천 알고리즘을 설계하고 구현합니다....

5. p_0260 (유사도: 0.5838)
   AI 기반 추천 시스템 개발 프로젝트. 사용자 행동을 분석하여 맞춤형 추천을 제공하는 AI 모델을 개발합니다....

6. p_1447 (유사도: 0.5810)
   AI 기반 추천시스템 개발 프로젝트. 사용자 맞춤형 추천을 위한 AI 알고리즘을 개발하고, 효율적인 데이터 처리를 통해 추천 정확도를 향상시키는...

7. p_0858 (유사도: 0.5807)
   AI 기반 추천 시스템 개발 프로젝트. 사용자의 행동 데이터를 분석하

To. 민서

평가 부분은 실제 시스템에 들어갈 필요가 없으니까 아래 코드는 사용하지 않아도 됩니당

1. '4단계 : 평가지표계산' 셀 (마지막에서 두번째)
2. '실행' 셀에서 #4단계, #평가지표계산 코드 (마지막 셀에서 앞과 같이 주석 달아져 있는 부분)

각각의 top-k 검색 수 / positive set

100 : 30 -> hit rate 0%

90~50 : 30 -> hit rate 40%

40 : 30 -> hit rate 20%

30 : 30 -> hit rate 20%

해석 : 프로젝트 데이터가 총 5000개인데, 그 중에서 컬럼별 유사도 상위 0.6%~1.9% 를 추출해서 가중치 합계를 통해 최종 5개를 선정할 때는 점차 hit rate가 상승하지만, 상위 2% 이상이 되는 순간 hit rate가 0%가 된다. (상위 레코드를 너무 많이 추출하면 분별력 하락)

앞으로 해볼 것 : 가중치 수정? (근데 한다면 뭘 어떻게 해야할지도 모르겠음..), positive set 구축 방법 수정? (어떻게?) 그냥 이대로?